In [1]:
!pip install torch torchvision

In [1]:
import torch
import torchvision
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [26]:
vgg = models.vgg16(pretrained=True)

In [27]:
for param in vgg.parameters():
    param.requires_grad = False

In [28]:
num_features = vgg.classifier[6].in_features
vgg.classifier[6] = nn.Linear(num_features, 10)

In [29]:
vgg = vgg.to(device)

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg.parameters(), lr=0.001, momentum=0.9)

In [31]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [32]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [33]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [34]:
num_epochs = 1
for epoch in range(num_epochs):
    vgg.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = vgg(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if i % 100: 
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 2.367126703262329
Epoch 1, Loss: 2.340350389480591
Epoch 1, Loss: 2.375511407852173
Epoch 1, Loss: 2.276177167892456
Epoch 1, Loss: 2.3992440700531006
Epoch 1, Loss: 2.3297581672668457
Epoch 1, Loss: 2.3252243995666504
Epoch 1, Loss: 2.3821680545806885
Epoch 1, Loss: 2.29646635055542
Epoch 1, Loss: 2.302363395690918
Epoch 1, Loss: 2.3299560546875
Epoch 1, Loss: 2.1785271167755127
Epoch 1, Loss: 2.151846170425415
Epoch 1, Loss: 2.1393215656280518
Epoch 1, Loss: 2.1477324962615967
Epoch 1, Loss: 2.111721992492676
Epoch 1, Loss: 2.0050203800201416
Epoch 1, Loss: 2.051914930343628
Epoch 1, Loss: 2.0380892753601074
Epoch 1, Loss: 2.0867507457733154
Epoch 1, Loss: 2.096113681793213
Epoch 1, Loss: 2.0769448280334473
Epoch 1, Loss: 1.9213632345199585
Epoch 1, Loss: 1.9350379705429077
Epoch 1, Loss: 1.9116398096084595
Epoch 1, Loss: 2.0025835037231445
Epoch 1, Loss: 1.8266576528549194
Epoch 1, Loss: 1.9725192785263062
Epoch 1, Loss: 1.86589515209198
Epoch 1, Loss: 1.7313153743743

KeyboardInterrupt: 

In [35]:
vgg.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = vgg(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {correct / total}")

Accuracy on test set: 0.6523
